# Introduction: Modeling

The last step in the machine learning pipeline is also where the value is derived. After we have developed the labels, each with its own cutoff time, we need to train a model to map from the features to predict the label. 

In this notebook, we will use the feature matrices and label times to train and test a machine learning model. We will work through a single prediction problem, predicting on the first of each month which customers will churn during the month. 

## Approach

Our basic machine learning approach is:

1. Prepare data for machine learning
    * Fill in missing values with median imputation
    * Encoding of categorical values
2. Split data into training and hold out testing based on time 
3. Evaluate a baseline logistic regression model
    * Also try a naive baseline for comparison
4. Try a non-linear more capable classifier, the Random Forest 
    * Use mostly default hyperparameters
    * Evaluate on hold-out testing data
5. Inspect predictions to determine if business need has been met
    * Precision recall curve used to tune threshold
    * Confusion matrix to assess predictions
6. Optimizer model automatically using an auto-ml library
    * Using TPOT although many options exist
    
The final outcome is an optimized model that solves the business problem of predicting customer churn with given parameters. The model can be deployed - used to make predictions on new data

In [1]:
import pandas as pd
import numpy as np

PARTITION_DIR = 's3://customer-churn-spark'

In [2]:
p0_fm = pd.read_csv(f'{PARTITION_DIR}/p0/MS-30_feature_matrix.csv')
p0_fm.head()

,msno,time,bd,city,registered_via,gender,SUM(logs.num_25),SUM(logs.num_50),SUM(logs.num_75),SUM(logs.num_985),...,WEEKEND(LAST(transactions.membership_expire_date)),DAY(LAST(logs.date)),DAY(LAST(transactions.transaction_date)),DAY(LAST(transactions.membership_expire_date)),MONTH(LAST(logs.date)),MONTH(LAST(transactions.transaction_date)),MONTH(LAST(transactions.membership_expire_date)),label,days_to_churn,churn_date
0,+9zx0+mA3IZQLyjmU88qbfqJ0q9okIfYZnDI6FqaN2o=,2015-01-01,0.0,1.0,7.0,NaN,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
1,+sZCvwt5NmFw4uE185pBid4cOxtXTHovIyPFqchulQg=,2015-01-01,0.0,1.0,7.0,NaN,0.0,0.0,0.0,0.0,...,1.0,NaN,1.0,1.0,NaN,1.0,2.0,NaN,395.0,NaN
2,+wzmLe86mMBeoIYoPedlt24WVTW6tabsRcaz81ZXBx0=,2015-01-01,0.0,1.0,7.0,NaN,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
3,/9+HJnqEryBbuH598zKqa8zb1Eypy927imqI9IWhJTk=,2015-01-01,29.0,8.0,9.0,male,3.0,0.0,0.0,0.0,...,0.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,488.0,NaN
4,/BAK3DkUpoUESh4t8qlWs16yop+sG3i3oPYDpv5uGI0=,2015-01-01,21.0,13.0,9.0,male,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN


### Retrieve Data

All of the data is stored in AWS S3. We'll retrieve 50 partitions of feature matrices.

In [3]:
def retrieve_data(partition_num, label_type='MS-30'):
    return pd.read_csv(f'{PARTITION_DIR}/p{partition_num}/{label_type}_feature_matrix.csv', low_memory=False)

The next cell retrieves a number of feature matrices for training and for testing. This is done in parallel using multiprocessing.

In [4]:
import random
random.seed(50)

from multiprocessing import Pool
pool = Pool(6)

# Set number of train and testing feature matrices
fms_to_get = 50

# Choose random sample of partitions
ps = random.sample(list(range(900)), fms_to_get)

# Retrieve feature matrices from S3
fms = []
for i, r in enumerate(pool.imap_unordered(retrieve_data, ps)):
    print(f'{round(100 * (i / fms_to_get), 2)}% complete.', end='\r')
    fms.append(r)
pool.close()
pool.join()

# Join together and drop rows with unknown label
feature_matrix = pd.concat(fms)
feature_matrix = feature_matrix[~feature_matrix['label'].isna()]
feature_matrix.shape

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:22: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



(1301613, 260)

The cell below converts the boolean types to integers for use in a machine learning model. Most of the boolean indicate whether or not all the values for customer were True (`ALL` primitive) or if the date was a weekend (`WEEKEND` primitive).

In [5]:
feature_matrix = feature_matrix.replace({'False': 0, 'True': 1})

In [6]:
feature_matrix_original = feature_matrix.copy()
feature_matrix.drop(columns = [c for c in ['churn', 'days_to_next_churn', 
                                           'churn_date'] if c in feature_matrix],
                    inplace = True)

bool_cols = [c for c in feature_matrix if 'ALL' in c or ('WEEKEND' in c and 'PERCENT_TRUE' not in c)]

for c in bool_cols:
    feature_matrix[c] = feature_matrix[c].astype(float)
feature_matrix[bool_cols].head()

,ALL(logs.WEEKEND(date)),ALL(transactions.WEEKEND(membership_expire_date) WHERE is_auto_renew = 0),ALL(transactions.WEEKEND(membership_expire_date) WHERE is_auto_renew = 1),ALL(transactions.WEEKEND(membership_expire_date) WHERE is_cancel = 0),ALL(transactions.WEEKEND(membership_expire_date) WHERE is_cancel = 1),ALL(transactions.WEEKEND(membership_expire_date)),ALL(transactions.WEEKEND(transaction_date) WHERE is_auto_renew = 0),ALL(transactions.WEEKEND(transaction_date) WHERE is_auto_renew = 1),ALL(transactions.WEEKEND(transaction_date) WHERE is_cancel = 0),ALL(transactions.WEEKEND(transaction_date) WHERE is_cancel = 1),...,ALL(transactions.is_cancel WHERE is_auto_renew = 0),ALL(transactions.is_cancel WHERE is_auto_renew = 1),ALL(transactions.is_cancel),LAST(logs.WEEKEND(date)),LAST(transactions.WEEKEND(membership_expire_date)),LAST(transactions.WEEKEND(transaction_date)),WEEKEND(LAST(logs.date)),WEEKEND(LAST(transactions.membership_expire_date)),WEEKEND(LAST(transactions.transaction_date)),WEEKEND(registration_init_time)
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,0.0,1.0
2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,0.0,1.0
3,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,0.0,0.0
4,NaN,NaN,0.0,0.0,NaN,0.0,NaN,0.0,0.0,NaN,...,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0


## Data Cleaning

We'll do a few basic data cleaning steps:

* Remove columns with many missing values
* Remove columns with a single unique value
* Remove highly correlated - colinear - columns

#### Missing Values

We'll drop any columns with more than 90% missing values.

In [7]:
missing_pct = feature_matrix.isnull().sum() / len(feature_matrix)
to_drop = list((missing_pct[missing_pct > 0.9]).index)
to_drop

['LAST(logs.seconds_per_song)',
 'MAX(logs.seconds_per_song)',
 'MEAN(logs.seconds_per_song)',
 'MIN(logs.seconds_per_song)',
 'STD(logs.seconds_per_song)',
 'SUM(logs.seconds_per_song)',
 'TOTAL_PREVIOUS_MONTH(logs.seconds_per_song, date)']

In [8]:
feature_matrix.drop(columns = to_drop, inplace = True)
feature_matrix.shape

(1301613, 252)

#### One Unique Value

Columns with only a single unique value contain no information and hence can be dropped.

In [9]:
one_unique = feature_matrix.apply(lambda x: x.nunique() == 1, axis = 0)
to_drop = list(one_unique[one_unique == True].index)
to_drop

['ALL(transactions.is_auto_renew WHERE is_cancel = 1)',
 'ALL(transactions.is_cancel WHERE is_auto_renew = 0)',
 'PERCENT_TRUE(transactions.is_cancel WHERE is_auto_renew = 0)']

In [10]:
feature_matrix.drop(columns = to_drop, inplace = True)
feature_matrix.shape

(1301613, 249)

#### Highly Correlated (collinear) Columns

Collinear columns can slow down training, lead to less interpretable models, and decrease generalization performance. Therefore, it's generally a good idea to remove one of each pair of highly correlated columns for machine learning. The following code identifies columns that exceed an absolute magnitude correlation of 0.95.

In [ ]:
threshold = 0.95

# Calculate correlations
corr_matrix = feature_matrix.corr().abs()

# Subset to the upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Identify names of columns with correlation above threshold
to_drop = [column for column in upper.columns if any(upper[column] >= threshold)]

In [ ]:
print(f'There are {len(to_drop)} columns to drop with correlation > {threshold}')

In [ ]:
feature_matrix.drop(columns = to_drop, inplace = True)
feature_matrix.shape

These data cleaning operations should increase the generalization performance of our model and make it more interpretable. A few simple operations can greatly improve a machine learning model and often are more effective than model optimization.

# Separate into Training and Testing Set

We'll separate into a training and testing set based on the date. We'll use 25% of the data in the testing and 75% in the training. Separating training and testing by the date is important in time sensitive problems because it prevents data leakage and gives a better estimate of the generalization performance of the model. On real data, our model will have to make forecasts of the future, and we can try to recreate that situation by using a hold-out set from later in time than the training data.

In [ ]:
feature_matrix['time'] = pd.to_datetime(feature_matrix['time'])
feature_matrix['time'].describe()

The cell below splits the data based on the time. First we sort by the time and then find the first 75% for training and the latter 25% for testing.

In [ ]:
train_fraction = 0.75
test_start = int(len(feature_matrix) * train_fraction)
feature_matrix.sort_values('time', inplace = True)

train = feature_matrix.iloc[:test_start].copy()
test = feature_matrix.iloc[test_start:].copy()

train.sort_values(['time'], inplace = True)
test.sort_values(['time'], inplace = True)
train.shape, test.shape

# Feature Preparation

The next blocks of code get the features ready for machine learning.


### Encoding Categoricals

First we need to one hot encode the features. After doing this, we align the training and testing dataframes so they have the same columns.

In [ ]:
train = pd.get_dummies(train.drop(columns = ['time', 'msno']))
test = pd.get_dummies(test.drop(columns = ['time', 'msno']))

train, test = train.align(test, join = 'inner', axis = 1)
train.shape, test.shape

### Extract the Labels

Now we can extract the labels. There are two different problems: one is a binary classification of whether or not the customer will churn during the month. The other is a regression: how many days are there until the next churn.

In [ ]:
y, test_y = np.array(train.pop('label')), np.array(test.pop('label'))

y_reg, test_y_reg = np.array(train.pop('days_to_churn')), np.array(
    test.pop('days_to_churn'))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.hist(y); plt.title('Label Distribution');
plt.ylabel('Count');

In [ ]:
plt.hist(np.nan_to_num(y_reg)); plt.title('Regression Labels');
plt.ylabel('Count'); plt.xlabel('Days to Next Churn');

### Fill in Missing Values

We can fill in missing values using the median of the column. As an important note, the missing test values are filled in with the median of the corresponding training feature.

In [ ]:
train = train.replace({np.inf: np.nan, -np.inf: np.nan}).\
    fillna(train.median()).fillna(0)

In [ ]:
test = test.replace({np.inf: np.nan, -np.inf: np.nan}).\
    fillna(train.median()).fillna(0)

In [ ]:
np.any(train.isnull()), np.any(np.isinf(train))

Now that there are no missing values and all of the values are numeric, our data is ready for machine learning. However, before we do machine learning, we need to figure out what a naive baseline would score.

# Naive Baseline

For a naive baseline, we can randomly guess that a customer has churned with the same frequence of the churns in the training data. We'll assess the predictions using a number of different metrics.

## Metrics

For an imbalanced classification problem, there are a number of metrics to consider:

* Receiver Operating Characteristic Area Under the Curve (ROC AUC): a measure between 0 and 1 comparing the performance of the classifier when predicting probabilities across a range of thresholds.
* Precision Score: number of true positives divided by the total number of positives predicted
* Recall Score: number of true positives divided by the total number of actual positives in the data
* F1 Score: Harmonic mean of precision and recall

The exact metric used and the threshold that our model needs to reach depends on the business need. We can tune the model to some extent to optimize for different metrics.

In [ ]:
np.random.seed(50)
naive_guess = np.random.binomial(1, p = np.mean(y), size = len(test_y))
naive_guess[:10], naive_guess.sum()

In [ ]:
from sklearn.metrics import (roc_auc_score, precision_score, 
                             recall_score, f1_score)

print(f'Naive Baseline\n')
roc = roc_auc_score(test_y, np.repeat(np.mean(y), len(test_y)))
print(f'ROC AUC: {round(roc, 4)}')

for metric in [precision_score, recall_score, f1_score]:
    print(f'{metric.__name__}: {round(metric(test_y, naive_guess), 4)}')


We can see that these metrics are very poor! With no machine learning, we are only able to identify 2.94% of the customer churns and 99.13% of our predicted churns are actually false positives.

In [ ]:
print(f'The percentage of churns is {100 * round(np.mean(y), 4)}% in the training data.')

# Machine Learning

Now we need to see if machine learning is up to the task of improving on these predictions. We'll start simple, using a linear model to assess if this problem is easy enough to be solved with Logistic Regression. 

(The machine learning models are implemented in [Scikit-Learn](https://sklearn.org/)). 

## Baseline Model

We can use a logistic regression in order to see baseline performance on this problem. If the logistic regression works well enough, then there is no need to move to a more complex model.

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state = 50, solver = 'lbfgs')

We can write a simple function to evaluate predictions. This implements the metrics used above. (A full list of metrics in Scikit-Learn can be found in [this documentation](http://scikit-learn.org/stable/modules/classes.html))

In [ ]:
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score


def evaluate(model, train, y, test, test_y):
    """Evaluate a machine learning model on four metrics:
       ROC AUC, precision score, recall score, and f1 score.
       
       Returns the model and the predictions."""
    
    model.fit(train, y)

    # Predict probabilities and labels
    probs = model.predict_proba(test)[:, 1]
    preds = model.predict(test)

    # Calculate ROC AUC
    roc = roc_auc_score(test_y, probs)
    name = repr(model).split('(')[0]
    print(f"{name}\n")
    print(f'ROC AUC: {round(roc, 4)}')

    # Iterate through metrics
    for metric in [precision_score, recall_score, f1_score]:
        # Use .__name__ attribute to list metric
        print(f'{metric.__name__}: {round(metric(test_y, preds), 4)}')

    return model, preds


model, preds = evaluate(model, train, y, test, test_y)

The baseline model does perform slightly better than guessing in terms of ROC AUC. The precision is slightly higher although the recall is much lower leading to an overall reduced f1 score (with a default threshold of 0.5 for classifying positive examples). This poor performance by the logistic regression indicates the problem of separating churn from not-churn is non-linear.

## More Complex Model

For a potentially better machine learning model, we can move to the [Random Forest Classifier](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html). From the results of the logistic regression, this looks to be a non-linear problem which means we should use a model capable of learning a non-linear decision boundary.

We'll use most of the default hyperparameters but alter a few to prevent overfitting. We can also set `class_weight = 'balanced'` to try and offset the impact of such an imbalanced classification problem.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, max_depth=40,
                               min_samples_leaf = 50,
                               n_jobs=-1, class_weight = 'balanced',
                               random_state = 50)

model, preds = evaluate(model, train, y, test, test_y)

The random forest performance is much better than just guessing! With no tuning, the model is able to identify 36% of the customer churns and the false positives have been reduced. This should give us confidence that we can solve this problem using machine learning.

# Model Validation

We need to inspect the model results to determine if it meets our business needs. This includes looking at the performance as well as the feature importances. We want to make sure that our model performs well, but also try and understand _why_ it performs well. 

## Precision Recall Curve

One of the best methods for tuning a model for a business need is through the precision recall curve. This shows the precision-recall tradeoff for different thresholds. Depending on the business requirement, we can change the threshold for classifying a positive example to alter the balance of true positives, false positives, false negatives, and true negatives. There will always be a tradeoff between precision and recall, but we can try to find the right balance by visually and quantitatively assessing the model.

In [ ]:
from sklearn.metrics import precision_recall_curve, confusion_matrix

import matplotlib.pyplot as plt
%matplotlib inline

plt.style.use('seaborn');

def plot_precision_recall(test_y, probs, title = 'Precision Recall Curve'):
    """Plot a precision recall curve for predictions. 
       Source: http://scikit-learn.org/stable/auto_examples/model_selection/plot_precision_recall.html#sphx-glr-auto-examples-model-selection-plot-precision-recall-py"""
    
    precision, recall, threshold = precision_recall_curve(test_y, probs)
    plt.figure(figsize = (8, 6))
    # In matplotlib < 1.5, plt.fill_between does not have a 'step' argument
    step_kwargs = ({'step': 'post'})
    plt.step(recall, precision, color='b', alpha=0.2,
             where='post')
    plt.fill_between(recall, precision, alpha=0.2, color='b', **step_kwargs)

    plt.xlabel('Recall', size = 18)
    plt.ylabel('Precision', size = 18)
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.title(title, size = 20)
    plt.xticks(size = 14); plt.yticks(size = 14)
    
    pr = pd.DataFrame({'precision': precision[:-1], 'recall': recall[:-1],
                       'threshold': threshold})
    return pr
    
probs = model.predict_proba(test)[:, 1]
pr_data = plot_precision_recall(test_y, probs, title = 'Precision-Recall Curve for Random Forest')

We can query the dataframe to find the threshold required for a given precision or recall. For example, to find the threshold for a precision of 50%, we use the following code:

In [ ]:
precision_above = pr_data.loc[pr_data['precision'] >= 0.5].copy()
precision_above.sort_values('recall', ascending = False, inplace = True)
precision_above.head()

At this threshold, our model must output a probability greater than 0.95 for an example to be classified as a positive churn. With this value, our model only identifies 0.11% of the actual churns as shown by the recall.

### Adjusting for the Business Requirement

Let's say we are required to have a recall of 50% in our model. This means our model finds 50% of the true churns in the data. We'll work through the rest of this notebook under this assumption. To find the threshold, we use:

In [ ]:
recall_above = pr_data.loc[pr_data['recall'] >= 0.5].copy()
recall_above.sort_values('precision', ascending = False, inplace = True)
recall_above.head()

In [ ]:
precision_attained = recall_above.iloc[0, 0]
threshold_required = recall_above.iloc[0, -1]

print(f'At a threshold of {round(threshold_required, 4)} the recall is 50% and the precision is {round(100 * precision_attained, 4)}%')

This means that in order to identify 50% of the actual churns, we'll have to accept that only 3.7% of the predicted positives are actually positive churns. 

## Confusion Matrix

A confusion matrix is often a useful way to visualize predictions. This shows the true values along the top row and the predicted values along the bottom row. Looking at the different cells, we can see where the model performed well and where it did not do so well.

We'll use the threshold identified above to construct the confusion matrix.

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.YlOrRd):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    Source: http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)
    plt.style.use('bmh')
    plt.figure(figsize = (9, 9))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, size = 22)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, size = 20)
    plt.yticks(tick_marks, classes, size = 20)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black",
                 size = 20)
    plt.grid(None)
    plt.ylabel('True label', size = 22)
    plt.xlabel('Predicted label', size = 22)
    plt.tight_layout()

In [ ]:
# Make predictions where probability is above threshold 
preds = np.zeros(len(test_y))
preds[probs >= threshold_required] = 1

# Make and plot confusion matrix
cm = confusion_matrix(test_y, preds)
plot_confusion_matrix(cm, classes = ['No Churn', 'Churn'],
                      title = 'Churn Confusion Matrix')

With our business requirement, this is the best prediction of what our performance would be on new data. The model is able to identiy 50% of churned customers compared to a baseline of around 3%. The precision has increased from the baseline 0.9% to 3.7%, a relative increase of over 300%. 

In [ ]:
abs(0.0087 - 0.037) / 0.0087

### Feature Importances

As one method for trying to understand how the model makes decisions, we can look at the most important features. The absolute value of the importances is not as useful as is the relative ranking of the features which is determined by how well the feature separates the classes when building the decision trees in the random forest.

In [ ]:
fi = pd.DataFrame({'importance': model.feature_importances_}, index=train.columns).\
    sort_values('importance', ascending=False)
fi.head()

In [ ]:
fi.iloc[:10]['importance'].plot.barh(color = 'r', edgecolor = 'k', 
                                     figsize = (14, 10), linewidth = 2)
ax = plt.gca()
ax.invert_yaxis();
plt.xticks(size = 20);
plt.yticks(size = 18)
plt.title('Most Important Features', size = 28);

To see why these features are important, we can plot the distribution colored by the value of the label.

In [ ]:
import seaborn as sns
plt.style.use('fivethirtyeight')

# Iterate through features
for feature in ['TIME_SINCE_LAST(transactions.transaction_date)',
                'AVG_TIME_BETWEEN(transactions.transaction_date)',
                'TOTAL_PREVIOUS_MONTH(transactions.daily_price, membership_expire_date)',
                'TOTAL_PREVIOUS_MONTH(transactions.actual_amount_paid, membership_expire_date)']:
    plt.figure(figsize = (16, 8))
    
    # Iterate through values of the label
    for label, grouped in train.groupby(y):
        # Plot the distribution of the feature
        sns.kdeplot(grouped[feature].dropna(), 
                    label = 'Churned' if label == 1 else 'No Churn');
    # Plot labeling
    plt.ylabel('Density'); plt.xlabel(f'{feature.capitalize()}')
    plt.title(f'Distribution of {feature.capitalize()}', size = 28);
    plt.legend(prop= {'size': 20});

There are significant differences between those customers who churned and those who did not. Most noticeably, the customers who churned had less activity in the previous month.

# Automated Model Optimization Using TPOT

If we are not pleased with the model results using an off-the-shelf algorithm from Scikit-Learn, there are a number of libraries for searching for the best model without any manual intervention. One of the easiest-to-se libraries is known as TPOT. This will search through hundreds of machine learning models, using evolutionary algorithms to guide the discovery process.

To evaluate the models, we'll use a TimeSeriesSplit. This makes three splits of the data based on the indexes so we need to ensure that our data is sorted by time (already done). 

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

tss = TimeSeriesSplit(n_splits = 3)

Now we make a `TPOTClassifier` object and pass in a few parameters. This particular use case will search 100 models, using `f1` scoring, the TimeSeriesSplit for cross validation, and taking advantage of all the cores on our machine.

In [ ]:
from tpot import TPOTClassifier

# Make tpot pipeline
tpot_pipeline = TPOTClassifier(generations = 10, population_size = 10, 
                               cv = tss, scoring = 'f1', 
                               n_jobs = -1, verbosity = 2,
                               random_state = 50)

In [ ]:
tpot_pipeline.fit(train, y)

In [ ]:
tpot_pipeline.fitted_pipeline_

In [ ]:
tpot_pipeline.export('best_pipeline.py')

## Best Pipeline

The following code shows the best pipeline exported by TPOT. We can just copy the code, train a model, and make predictions. 

```python
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler

# NOTE: Make sure that the class is labeled 'target' in the data file
tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR', dtype=np.float64)
features = tpot_data.drop('target', axis=1).values
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data['target'].values, random_state=None)

# Average CV score on the training set was:0.4381
exported_pipeline = make_pipeline(
    MinMaxScaler(),
    GradientBoostingClassifier(learning_rate=0.01, max_depth=7, max_features=0.75, min_samples_leaf=12, min_samples_split=3, n_estimators=100, subsample=0.05)
)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)
```

In [ ]:
# from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import MinMaxScaler

# # Average CV score on the training set was:0.43811110082970917
# exported_pipeline = make_pipeline(
#     MinMaxScaler(),
#     GradientBoostingClassifier(learning_rate=0.01, max_depth=7, max_features=0.75, 
#                                min_samples_leaf=12, min_samples_split=3, n_estimators=100, subsample=0.05)
# )

In [ ]:
model, preds = evaluate(exported_pipeline, train, y, test, test_y)

In [ ]:
cm = confusion_matrix(test_y, preds)
plot_confusion_matrix(cm, classes = ['No Churn', 'Churn'], title = 'Optimized Model Confusion Matrix')

In [ ]:
probs = model.predict_proba(test)[:, 1]
plot_precision_recall(test_y, probs, title = 'Precision-Recall Curve for Optimized Model')